In [1]:
import mysql.connector
import requests

In [2]:
conn_relacional = mysql.connector.connect(
    host="127.0.0.1",
    port=3306,
    user="root",
    password="0101",
    database="relacional",
)

conn_dimensional = mysql.connector.connect(
    host="127.0.0.1",
    port=3306,
    user="root",
    password="0101",
    database="dimensional",
)

cursor_relacional = conn_relacional.cursor()
cursor_dimensional = conn_dimensional.cursor()

In [3]:
def pais_para_continente(pais):
    url = f"https://restcountries.com/v3.1/translation/{pais.lower()}"
    response = requests.get(url)

    if response.status_code != 200:
        return None
    
    data = response.json()[0]
    continente = data['region']

    traducao = {
        'Africa': 'África',
        'Americas': 'América',
        'Europe': 'Europa',
        'Asia': 'Ásia',
        'Oceania': 'Oceania',
        'Antarctic': 'Antártida'
    }
    return traducao.get(continente, continente)

## Localidade

In [ ]:
cursor_relacional.execute(
    "SELECT idPais, nmPais FROM pais"
)
paises = cursor_relacional.fetchall()

cursor_relacional.execute(
    "SELECT idEstado, nmEstado FROM estado"
)
estados = cursor_relacional.fetchall()

cursor_relacional.execute(
    "SELECT idCidade, nmCidade FROM cidade"
)
cidades = cursor_relacional.fetchall()

continentes = []
for id_pais, nome_pais in paises:
    continentes.append(pais_para_continente(nome_pais))
    
# Inserts
cursor_dimensional.executemany(
    "INSERT INTO d_localidade (idLocalidade, pais, estado, cidade, continente) VALUES (%s, %s, %s, %s, %s)",
    [(i+1, paises[0][1], estados[i][1], cidades[i][1], continentes[0]) for i in range(len(estados))],
)

NameError: name 'cursor_relacional' is not defined

## Usuário

In [24]:
from datetime import datetime

def get_years(date, actual_date):
    return actual_date.year - date.year + 1

In [25]:
from datetime import datetime

cursor_relacional.execute(
    "SELECT idUsuario, nome, dtNascimento FROM usuario",
)
usuarios = cursor_relacional.fetchall()

actual_date = datetime.now().date()
dados = []
for id_usuario, nome, dt_nascimento in usuarios:
    faixa_etaria = get_years(dt_nascimento, actual_date) // 10
    dados.append((id_usuario, nome, faixa_etaria))

cursor_dimensional.executemany(
    "INSERT INTO d_usuario (idUsuario, nome, faixa_etaria) VALUES (%s, %s, %s)",
    dados,
)

## Pagamento

In [29]:
# campos: idPagamento, vlrRecebido
cursor_relacional.execute(
    "SELECT idPagamento, vlrRecebido FROM pagamento",
)
pagamentos = cursor_relacional.fetchall()

cursor_dimensional.executemany(
    "INSERT INTO d_Pagamento (idPagamento, vlrRecebido) VALUES (%s,%s)",
    pagamentos,
)

## Plano

In [ ]:
cursor_relacional.execute(
    "SELECT idPlano, nmPlano FROM plano",
)
planos = cursor_relacional.fetchall()

cursor_dimensional.executemany(
    "INSERT INTO d_plano (idPlano, nmPlano) VALUES (%s, %s)",
    planos
)

[(1, 'Normal'), (2, 'Normal+'), (3, 'Premium'), (4, 'Premium+'), (5, 'PremiumPlus+')]


## Assinatura

In [ ]:
cursor_relacional.execute(
    "SELECT idAssinatura, Usuario_idUsuario, dtInicio FROM assinatura"
)
assinaturas = cursor_relacional.fetchall()

for id_assinatura, id_usuario, dt_inicio in assinaturas:
    # TODO: Tentar refazer usando dados já inseridos no banco dimensional
    cursor_relacional.execute(
        "SELECT pais_idPais, cidade_idCidade, estado_idEstado FROM usuario WHERE idUsuario=%s",
        (id_usuario,)
    )
    usuario = cursor_relacional.fetchone()
    pais_id, cidade_id, estado_id = usuario

    cursor_relacional.execute(
        "SELECT nmPais FROM pais WHERE idPais=%s",
        (pais_id,)
    )
    pais = cursor_relacional.fetchone()[0]

    cursor_relacional.execute(
        "SELECT nmEstado FROM estado WHERE idEstado=%s",
        (estado_id,)
    )
    estado = cursor_relacional.fetchone()[0]

    cursor_relacional.execute(
        "SELECT nmCidade FROM cidade WHERE idCidade=%s",
        (cidade_id,)
    )
    cidade = cursor_relacional.fetchone()[0]

    cursor_dimensional.execute(
        """
        SELECT idLocalidade FROM d_localidade
        WHERE pais=%s AND estado=%s AND cidade=%s
        """,
        (pais, estado, cidade)
    )
    localidade = cursor_dimensional.fetchone()[0]

    cursor_dimensional.execute(
        """
        SELECT idData FROM d_data
        WHERE mes=%s AND ano=%s
        """,
        (dt_inicio.month, dt_inicio.year)
    )
    data = cursor_dimensional.fetchall()
    print(dt_inicio.month, dt_inicio.year, len(data))

pais: Brasil, estado: Roraima, cidade: Gonçalves
3 2013 31
pais: Brasil, estado: Maranhão, cidade: Carvalho de Cavalcante
2 2015 28
pais: Brasil, estado: Rio Grande do Norte, cidade: Pereira
8 2014 31
pais: Brasil, estado: Tocantins, cidade: Almeida do Oeste
1 2013 31
pais: Brasil, estado: Mato Grosso do Sul, cidade: da Paz Paulista
4 2015 30


In [ ]:
# conn_dimensional.commit()
